In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import scipy.io as sio
from keras.utils import np_utils

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, cohen_kappa_score
from sklearn.decomposition import PCA

from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Reshape, Dropout
from keras.layers.normalization import BatchNormalization
from keras import optimizers

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping

import cv2
from skimage.util import view_as_windows

In [2]:
from config.utils import *
from config.models import *

In [3]:
######### GLOBAL PARAMETERS #########
dataset = 'IP'
testSize = 0.75
windowSize = 25
K = 30 if dataset == 'IP' else 15
EPOCHS = 20
######### GLOBAL PARAMETERS #########

In [4]:
X, Y = loadData(dataset)

In [5]:
units, target_names = targets(dataset)

In [6]:
X.shape

(145, 145, 200)

In [7]:
Y.shape

(145, 145)

In [8]:
X,pca = applyPCA(X,N=K)
ev=pca.explained_variance_ratio_
print('X.shape',X.shape)
print('y.shape',Y.shape)

X.shape (145, 145, 30)
y.shape (145, 145)


In [9]:
X , Y = createImageCubes(X,Y,windowSize=windowSize)

In [10]:
print('X.shape',X.shape)
print('y.shape',Y.shape)

X.shape (10249, 25, 25, 30)
y.shape (10249,)


In [11]:
X_Train , X_Test , Y_Train, Y_Test = train_test_split(X,Y,test_size=testSize,random_state=345)

In [12]:
Y_Train = np_utils.to_categorical(Y_Train)
Y_Train.shape

(2562, 16)

In [13]:
X_Train = X_Train.reshape(-1, windowSize, windowSize, K, 1)
X_Train.shape

(2562, 25, 25, 30, 1)

In [14]:
X_Test = X_Test.reshape(-1, windowSize, windowSize, K, 1)
X_Test.shape

(7687, 25, 25, 30, 1)

In [15]:
Y_Test = np_utils.to_categorical(Y_Test)
Y_Test.shape

(7687, 16)

In [16]:
X_Train.shape

(2562, 25, 25, 30, 1)

In [17]:
Model = make_3d_1(units, dataset)

In [18]:
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 23, 23, 26, 8)     368       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 21, 21, 24, 16)    3472      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 10, 10, 12, 16)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 10, 10, 12, 16)    64        
_________________________________________________________________
reshape (Reshape)            (None, 10, 10, 192)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 8, 8, 64)          110656    
_________________________________________________________________
flatten (Flatten)            (None, 4096)              0

In [19]:
adam = Adam(lr=0.001, decay=1e-06)
Model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath = "first" + dataset + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=2)

callbacks_list = [checkpoint]

In [1]:
History = Model.fit(x=X_Train, y=Y_Train,epochs=EPOCHS, batch_size = 128, callbacks = callbacks_list)

In [22]:
Y_Pred = Model.predict(X_Test)
Y_Pred = np.argmax(Y_Pred, axis=1)

In [23]:
Report = (classification_report(np.argmax(Y_Test, axis=1), Y_Pred, target_names=target_names))
print(Report)

Matrix = (confusion_matrix(np.argmax(Y_Test, axis=1), Y_Pred))

                              precision    recall  f1-score   support

                     Alfalfa       1.00      0.97      0.98        30
                 Corn-notill       0.99      0.87      0.93      1087
                Corn-mintill       1.00      0.89      0.94       615
                        Corn       0.99      0.99      0.99       185
               Grass-pasture       0.96      0.91      0.93       386
                 Grass-trees       0.88      1.00      0.93       547
         Grass-pasture-mowed       0.00      0.00      0.00        22
               Hay-windrowed       0.98      1.00      0.99       366
                        Oats       0.00      0.00      0.00        15
              Soybean-notill       0.98      0.86      0.92       720
             Soybean-mintill       0.89      0.98      0.93      1824
               Soybean-clean       0.93      0.95      0.94       439
                       Wheat       0.89      1.00      0.94       154
                   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
Model = make_3d_2(units, dataset)

In [26]:
Model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 23, 23, 24, 8)     512       
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 21, 21, 20, 16)    5776      
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 19, 19, 18, 16)    6928      
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 17, 17, 16, 16)    6928      
_________________________________________________________________
batch_normalization_1 (Batch (None, 17, 17, 16, 16)    64        
_________________________________________________________________
reshape_1 (Reshape)          (None, 17, 17, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 15, 15, 64)       

In [27]:
adam = Adam(lr=0.001, decay=1e-06)
Model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

filepath = "second" + dataset + ".hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
early_stop = EarlyStopping(monitor='val_loss',patience=2)

callbacks_list = [checkpoint]

In [28]:
History = Model.fit(x=X_Train, y=Y_Train,epochs=EPOCHS, batch_size = 128, callbacks = callbacks_list)

Epoch 1/17
21/21 [==============================] - ETA: 0s - loss: 2.8709 - accuracy: 0.1432
Epoch 00001: accuracy improved from -inf to 0.14325, saving model to secondIP.hdf5
21/21 [==============================] - 42s 2s/step - loss: 2.8709 - accuracy: 0.1432
Epoch 2/17
21/21 [==============================] - ETA: 0s - loss: 2.2971 - accuracy: 0.2541
Epoch 00002: accuracy improved from 0.14325 to 0.25410, saving model to secondIP.hdf5
21/21 [==============================] - 39s 2s/step - loss: 2.2971 - accuracy: 0.2541
Epoch 3/17
21/21 [==============================] - ETA: 0s - loss: 2.0251 - accuracy: 0.3314
Epoch 00003: accuracy improved from 0.25410 to 0.33138, saving model to secondIP.hdf5
21/21 [==============================] - 39s 2s/step - loss: 2.0251 - accuracy: 0.3314
Epoch 4/17
21/21 [==============================] - ETA: 0s - loss: 1.8416 - accuracy: 0.3903
Epoch 00004: accuracy improved from 0.33138 to 0.39032, saving model to secondIP.hdf5
21/21 [===============

In [30]:
Y_Pred = Model.predict(X_Test)
Y_Pred = np.argmax(Y_Pred, axis=1)

In [31]:
Report = (classification_report(np.argmax(Y_Test, axis=1), Y_Pred, target_names=target_names))
print(Report)

Matrix = (confusion_matrix(np.argmax(Y_Test, axis=1), Y_Pred))

                              precision    recall  f1-score   support

                     Alfalfa       0.00      0.00      0.00        30
                 Corn-notill       0.86      0.89      0.88      1087
                Corn-mintill       0.87      1.00      0.93       615
                        Corn       1.00      0.18      0.31       185
               Grass-pasture       0.81      0.99      0.89       386
                 Grass-trees       0.86      0.99      0.92       547
         Grass-pasture-mowed       0.00      0.00      0.00        22
               Hay-windrowed       1.00      0.99      0.99       366
                        Oats       0.00      0.00      0.00        15
              Soybean-notill       0.98      0.96      0.97       720
             Soybean-mintill       0.99      0.98      0.98      1824
               Soybean-clean       0.69      0.97      0.81       439
                       Wheat       0.20      0.01      0.01       154
                   